In [35]:
# get max test id so far
import os
test_ids = [int(f.split("_")[1].split(".")[0]) for f in os.listdir("parsed_tests") if f.startswith("test_")]

max_test_id = max(test_ids)
max_test_id

967750

In [50]:
# Krok 1: Instalace potřebných knihoven
# Odkomentujte a spusťte následující řádek, pokud knihovny ještě nemáte nainstalované
# !pip install requests pdfplumber

import requests
import io
import pdfplumber
import re
import json
import os
from pprint import pprint

def analyzuj_test_z_pdf(url: str):
    """
    Stáhne PDF test z dané URL, vyparsuje z něj otázky a odpovědi
    pomocí robustní metody, která zvládá víceřádkové otázky a odstraňuje patičky.
    """
    print(f"Stahuji PDF z adresy: {url}...")

    try:
        # Stažení PDF souboru s hlavičkou pro obejití základní ochrany
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, timeout=30, headers=headers)
        response.raise_for_status()
        print("PDF úspěšně staženo.")

        pdf_content = response.content

        # Extrakce textu z PDF
        print("Parsuji obsah PDF...")
        full_text = ""
        with io.BytesIO(pdf_content) as pdf_stream:
            with pdfplumber.open(pdf_stream) as pdf:
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

        if "Přezkušovací test ULL Pilot" not in full_text:
            print("PDF neobsahuje očekávaný text, zřejmě se nejedná o ULL test nebo stránka neexistuje.")
            return None

        # --- NOVÝ KROK: ČIŠTĚNÍ TEXTU OD PATIČEK ---
        # Odstraníme všechny řádky, které začínají "Tisk:", abychom se zbavili patiček.
        full_text = re.sub(r'^Tisk:.*$', '', full_text, flags=re.MULTILINE)
        # ---------------------------------------------

        print("Extrakce textu dokončena. Zpracovávám otázky...")

        pattern_otazky = re.compile(r'(\d+)\.\s+([\s\S]+?)Počet bodů:\s*(\d+)', re.MULTILINE)
        otazky = []
        matches = pattern_otazky.finditer(full_text)

        for match in matches:
            blok_otazky = match.group(2).strip()
            body_otazky = int(match.group(3))

            prvni_odpoved_match = re.search(r'\n\s*[A-C]\.', blok_otazky)

            if prvni_odpoved_match:
                index_odpovedi = prvni_odpoved_match.start()
                text_otazky_blok = blok_otazky[:index_odpovedi].strip()
                odpovedi_blok = blok_otazky[index_odpovedi:].strip()
            else:
                text_otazky_blok = blok_otazky
                odpovedi_blok = ""

            text_otazky = ' '.join(text_otazky_blok.split())

            moznosti = {}
            spravna_odpoved = None

            # Regex pro nalezení jednotlivých odpovědí v bloku
            # Hledá písmeno, tečku, a pak vše až do dalšího písmene s tečkou na začátku řádku nebo do konce bloku
            pattern_odpovedi = re.compile(r'([A-C])\.\s*([\s\S]*?)(?=\n[A-C]\.|\Z)', re.MULTILINE)
            odpovedi_matches = pattern_odpovedi.finditer(odpovedi_blok)

            for odpoved_match in odpovedi_matches:
                pismeno = odpoved_match.group(1)
                text_odpovedi_surovy = odpoved_match.group(2)

                if '☺' in text_odpovedi_surovy or '☻' in text_odpovedi_surovy:
                    spravna_odpoved = pismeno

                cisty_text = re.sub(r'[x☺☻●]', '', text_odpovedi_surovy).strip()
                moznosti[pismeno] = ' '.join(cisty_text.split())

            otazky.append({
                "text_otazky": text_otazky,
                "moznosti": moznosti,
                "spravna_odpoved": spravna_odpoved,
                "body": body_otazky
            })

        datum_match = re.search(r'Datum\s+([\d.]+)', full_text)
        datum_testu = datum_match.group(1) if datum_match else "N/A"

        print("Zpracování dokončeno.")
        return {
            "datum_testu": datum_testu,
            "prehled_otazek": otazky,
        }

    except requests.exceptions.HTTPError as e:
        print(f"Test na adrese {url} pravděpodobně neexistuje (HTTP chyba: {e.response.status_code}).")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Chyba při stahování PDF: {e}")
        return None
    except Exception as e:
        print(f"Nastala neočekávaná chyba při parsování: {e}")
        return None

# --- Spouštěcí část skriptu (vaše smyčka) ---

if not os.path.exists("parsed_tests"):
    os.makedirs("parsed_tests")
    print("Vytvořena složka 'parsed_tests'")

url_base = "https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id="
start_id = 950000
end_id = 967769

chyba_count = 0
max_chyb = 50

for i in range(start_id, end_id + 1):
    nazev_souboru = f"test_{i}.json"
    cesta_k_souboru = os.path.join("parsed_tests", nazev_souboru)

    if os.path.exists(cesta_k_souboru):
        print(f"Test s ID {i} již existuje, přeskakuji.")
        chyba_count = 0
        continue

    print(f"\n--- Zpracovávám test s ID {i} ---")
    url_pdf_testu = url_base + str(i)

    zpracovana_data = analyzuj_test_z_pdf(url_pdf_testu)

    if zpracovana_data is None or not zpracovana_data.get("prehled_otazek"):
        print(f"Test s ID {i} se nepodařilo zpracovat nebo je prázdný. Ukládám prázdný soubor.")
        with open(cesta_k_souboru, "w") as f:
            json.dump({}, f)
        chyba_count += 1
        if chyba_count >= max_chyb:
            print(f"Počet po sobě jdoucích chyb dosáhl {max_chyb}, ukončuji zpracování.")
            break
        continue

    chyba_count = 0

    print(f"Úspěšně zpracováno {len(zpracovana_data['prehled_otazek'])} otázek. Ukládám do souboru {nazev_souboru}.")
    with open(cesta_k_souboru, "w", encoding='utf-8') as f:
        json.dump(zpracovana_data, f, ensure_ascii=False, indent=4)

print("\n--- Zpracování všech ID dokončeno. ---")

Test s ID 950000 již existuje, přeskakuji.
Test s ID 950001 již existuje, přeskakuji.
Test s ID 950002 již existuje, přeskakuji.
Test s ID 950003 již existuje, přeskakuji.
Test s ID 950004 již existuje, přeskakuji.
Test s ID 950005 již existuje, přeskakuji.
Test s ID 950006 již existuje, přeskakuji.
Test s ID 950007 již existuje, přeskakuji.
Test s ID 950008 již existuje, přeskakuji.
Test s ID 950009 již existuje, přeskakuji.
Test s ID 950010 již existuje, přeskakuji.
Test s ID 950011 již existuje, přeskakuji.
Test s ID 950012 již existuje, přeskakuji.
Test s ID 950013 již existuje, přeskakuji.
Test s ID 950014 již existuje, přeskakuji.
Test s ID 950015 již existuje, přeskakuji.

--- Zpracovávám test s ID 950016 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=950016...
PDF úspěšně staženo.
Parsuji obsah PDF...
PDF neobsahuje očekávaný text, zřejmě se nejedná o ULL test nebo stránka neexistuje.
Test s ID 950016 se nepodařilo zpracovat

KeyboardInterrupt: 

In [1]:
# Krok 1: Instalace potřebných knihoven
# Odkomentujte a spusťte následující řádek, pokud knihovny ještě nemáte nainstalované
# !pip install requests pdfplumber tqdm

import requests
import io
import pdfplumber
import re
import json
import os
import time
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ==============================================================================
# ČÁST 1: FUNKCE PRO PARSOVÁNÍ (zůstává téměř stejná)
# ==============================================================================

def analyzuj_test_z_pdf(pdf_content: bytes):
    """
    Analyzuje obsah PDF a extrahuje data testu.
    Tato funkce je volána z workeru. Vrací data nebo None.
    """
    try:
        full_text = ""
        with io.BytesIO(pdf_content) as pdf_stream:
            with pdfplumber.open(pdf_stream) as pdf:
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

        if "Přezkušovací test ULL Pilot" not in full_text:
            return None # Není to platný test

        # Odstranění patiček pro čistší parsování
        full_text = re.sub(r'^Tisk:.*$', '', full_text, flags=re.MULTILINE)

        pattern_otazky = re.compile(r'(\d+)\.\s+([\s\S]+?)Počet bodů:\s*(\d+)', re.MULTILINE)
        otazky = []
        matches = pattern_otazky.finditer(full_text)

        for match in matches:
            blok_otazky = match.group(2).strip()
            body_otazky = int(match.group(3))

            prvni_odpoved_match = re.search(r'\n\s*[A-C]\.', blok_otazky)

            if prvni_odpoved_match:
                index_odpovedi = prvni_odpoved_match.start()
                text_otazky_blok = blok_otazky[:index_odpovedi].strip()
                odpovedi_blok = blok_otazky[index_odpovedi:].strip()
            else:
                text_otazky_blok = blok_otazky
                odpovedi_blok = ""

            text_otazky = ' '.join(text_otazky_blok.split())

            moznosti = {}
            spravna_odpoved = None

            pattern_odpovedi = re.compile(r'([A-C])\.\s*([\s\S]*?)(?=\n[A-C]\.|\Z)', re.MULTILINE)
            odpovedi_matches = pattern_odpovedi.finditer(odpovedi_blok)

            for odpoved_match in odpovedi_matches:
                pismeno = odpoved_match.group(1)
                text_odpovedi_surovy = odpoved_match.group(2)

                if '☺' in text_odpovedi_surovy or '☻' in text_odpovedi_surovy:
                    spravna_odpoved = pismeno

                cisty_text = re.sub(r'[x☺☻●]', '', text_odpovedi_surovy).strip()
                moznosti[pismeno] = ' '.join(cisty_text.split())

            otazky.append({
                "text_otazky": text_otazky,
                "moznosti": moznosti,
                "spravna_odpoved": spravna_odpoved,
                "body": body_otazky
            })

        datum_match = re.search(r'Datum\s+([\d.]+)', full_text)
        datum_testu = datum_match.group(1) if datum_match else "N/A"

        return {
            "datum_testu": datum_testu,
            "prehled_otazek": otazky,
        }
    except Exception:
        # Pokud parsování selže, vrátíme None
        return None

# ==============================================================================
# ČÁST 2: FUNKCE PRO JEDNOHO WORKERA (nová funkce)
# ==============================================================================

def zpracuj_jeden_test(test_id):
    """
    Kompletní logika pro zpracování jednoho ID: kontrola existence, stažení,
    parsování a uložení souboru. Vrací stav zpracování.
    """
    url_base = "https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id="
    vystupni_slozka = "parsed_tests"

    nazev_souboru = f"test_{test_id}.json"
    cesta_k_souboru = os.path.join(vystupni_slozka, nazev_souboru)

    if os.path.exists(cesta_k_souboru):
        return "skipped" # Přeskočeno

    url_pdf = url_base + str(test_id)

    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url_pdf, timeout=15, headers=headers)

        # Pokud stránka neexistuje (404) nebo jiná chyba, rovnou selžeme
        if response.status_code != 200:
            with open(cesta_k_souboru, "w") as f: json.dump({}, f)
            return "failed"

        # Zpracování PDF obsahu
        zpracovana_data = analyzuj_test_z_pdf(response.content)

        # Uložení výsledku (i neúspěšného parsování)
        if zpracovana_data and zpracovana_data.get("prehled_otazek"):
            with open(cesta_k_souboru, "w", encoding='utf-8') as f:
                json.dump(zpracovana_data, f, ensure_ascii=False, indent=4)
            return "success"
        else:
            with open(cesta_k_souboru, "w") as f: json.dump({}, f)
            return "failed"

    except requests.exceptions.RequestException:
        # Chyba sítě/timeoutu
        with open(cesta_k_souboru, "w") as f: json.dump({}, f)
        return "failed"

# ==============================================================================
# ČÁST 3: HLAVNÍ SPOUŠTĚCÍ BLOK
# ==============================================================================

if __name__ == "__main__":
    # --- Nastavení ---
    start_id = 950000
    end_id = 967769
    vystupni_slozka = "parsed_tests"
    # Použijeme všechna dostupná jádra CPU, ale maximálně např. 16, abychom nezahltili server
    pocet_workeru = 4

    # Vytvoření složky pro výsledky, pokud neexistuje
    if not os.path.exists(vystupni_slozka):
        os.makedirs(vystupni_slozka)
        print(f"Vytvořena složka '{vystupni_slozka}'")

    # Seznam všech ID, které chceme zpracovat
    vsechna_id = range(start_id, end_id + 1)

    print(f"Spouštím zpracování {len(vsechna_id)} testů na {pocet_workeru} jádrech CPU.")

    # Vytvoření poolu workerů
    with Pool(processes=pocet_workeru) as pool:
        # Použijeme pool.imap_unordered pro efektivní zpracování s progress barem
        # imap_unordered vrací výsledky, jakmile jsou hotové, což je ideální pro TQDM
        results = list(tqdm(pool.imap_unordered(zpracuj_jeden_test, vsechna_id), total=len(vsechna_id)))

    # Vyhodnocení výsledků
    uspesne = results.count("success")
    preskoceno = results.count("skipped")
    neúspěšné = results.count("failed")

    print("\n--- Zpracování dokončeno ---")
    print(f"Celkem úspěšně zpracováno: {uspesne}")
    print(f"Celkem přeskočeno (již existovalo): {preskoceno}")
    print(f"Celkem neúspěšně (chyba nebo neexistuje): {neúspěšné}")

Spouštím zpracování 17770 testů na 4 jádrech CPU.


100%|██████████| 17770/17770 [01:01<00:00, 287.64it/s]  


--- Zpracování dokončeno ---
Celkem úspěšně zpracováno: 310
Celkem přeskočeno (již existovalo): 14861
Celkem neúspěšně (chyba nebo neexistuje): 2599
